# ppg.ipynb
Functions to examine rolling correlations between device sensor outputs.

Authors:  
– Jon Clucas, 2017 jon.clucas@childmind.org  
– Arno Klein, 2017  
© 2017, Child Mind Institute, Apache v2.0 License

setup:

In [ ]:
%matplotlib inline
from annotate_range import annotation_line
from astropy.stats import median_absolute_deviation as mad
from chart_data import bland_altman_plot, df_devices, linechart, write_csv
from config import short_dir, test_urls
from datetime import datetime, timedelta
from matplotlib.dates import DateFormatter
from normalize_acc_data import actigraph_acc, geneactiv_acc
from plot_normalized_vector_lengths import baseshift_and_renormalize
from utilities.fetch_data import fetch_check_data, fetch_data, fetch_hash
import json, matplotlib as mpl, numpy as np, os, pandas as pd, matplotlib.pyplot as plt
"""
with open(os.path.join('./line_charts/device_colors.json')) as fp:
    color_key = json.load(fp)
"""
pd.set_option('mode.use_inf_as_null', True)
ppg_hashes = {'Wavelet_ppg': '1af7572640aea22a0af354bedbeb7a1e',
              'e4_ppg': '74a4074802c03fa22fb3ae8a6715263b'}
if not os.path.exists('./sample_data'):
    os.makedirs('./sample_data')

### PPG
E4 has a 'Timestamp' column and a combined red and green data column.
Wavelet has a 'Timestamp' column and four data columns: red, infrared, filtered red, and filtered infrared

In [3]:
for ppg in ['e4_ppg', 'Wavelet_ppg']:
    try:
        fetch_check_data(ppg, test_urls()[ppg], ppg_hashes, cache_directory='./sample_data', append='.csv',
                         verbose=True)
    except:
        hashes[ppg] = fetch_hash(fetch_data(test_urls()[ppg], os.path.join('./sample_data', ppg), '.csv'))
ppg_dfs = {}
for ppg in ['e4_ppg', 'Wavelet_ppg']:
    ppg_df[ppg] = pd.read_csv(fetch_check_data(ppg, test_urls()[ppg], ppg_hashes, cache_directory='./sample_data',
                  append='.csv'), parse_dates=['Timestamp'], infer_datetime_format=True)
    print(str(list(ppg_df[ppg].columns)))

NameError: name 'ppg_df' is not defined